# frame_extraction
We've been able to break the film down into individual image frames and perform some basic cinematic analyses on them. We've stored this information/metadata in dataframes. We'll want to be able to use these analyses to select specific image frames for future model training purposes, such as selecting all images with a specific color palette or with a certain number of characters present.

For the purposes of training CNNs and GANs with Keras/TensorFlow, we'll want to copy frames images into a single folder. We can use `os` and other Python libraries to navigate the directory structure and copy files around.

In [1]:
import os
from distutils.dir_util import copy_tree
from shutil import copy, move, rmtree
import pandas as pd

Each movie's frames are stored in their own directory in `/frame_per_second`.

In [2]:
%ls ../frame_per_second/

all_is_true_2018/
before_sunset_2004/
black_and_blue_2019/
booksmart_2019/
extremely_wicked_shockingly_evil_and_vile_2019/
ford_v_ferrari_2019/
joker_2019/
knives_out_2019/
lost_in_translation_2003/
oceans_eleven_2001/
once_upon_a_time_in_hollywood_2019/
plus_one_2019/
second_act_2018/
the_hustle_2019/
toy_story_4_2019/


### Medium Close-Ups
Let's say we want to compile movie frames where it's a close-up of a single-character. Most two-character dialogue scenes make extensive use of medium close-ups of each character. 

We'll start with *The Hustle* (2019).

In [3]:
import sys
sys.path.append('../data_serialization')
from serialization_preprocessing_io import *

In [4]:
film = 'the_hustle_2019'
srt_df, subtitle_df, sentence_df, vision_df, face_df = read_pickle(film)

First we look at frames with prim_char_flag equal to 1. These are frames that satisfy one of the two conditions:
- There is one face in frame
- There are multiple face in frame, but there is one face much larger than the others (the other faces are 75% or less in size)

In [5]:
face_df[face_df['prim_char_flag'] == 1][100:104]

,face_locations,face_sizes,face_encodings,faces_found,prim_char_flag,p_center_alignment,p_horizontal_distance,p_open_mouth,p_face_cluster,p_emotion
frame,,,,,,,,,,
194,"[(102, 547, 263, 387)]",[8.37],"[[-0.1602196991443634, 0.05712738633155823, 0....",1,1,None,NaN,1.0,28.0,surprise
195,"[(102, 405, 263, 244)]",[8.48],"[[-0.1326746940612793, 0.05337563902139664, 0....",1,1,left,183.0,1.0,10.0,fear
196,"[(100, 366, 233, 233)]",[5.79],"[[-0.0987188071012497, 0.0743609368801117, 0.0...",1,1,left,194.0,1.0,6.0,surprise
197,"[(66, 654, 227, 494)]",[8.37],"[[-0.18016411364078522, 0.09841547906398773, 0...",1,1,right,227.0,0.0,28.0,surprise


In [6]:
len(face_df[face_df['prim_char_flag'] == 1])

3163

Next, we'll want to make sure the sizes of the faces are at least a certain percentage of the overall frame size. We'll set this threshold at 4%. There's a bit of difficulty using the traditional Pandas filters, because we need to process the first element of `face_sizes`, which is a list.

In [7]:
face_df.loc[194].face_sizes

[8.37]

In [8]:
face_df.loc[194].face_sizes[0]

8.37

In [9]:
prim_char_indices = face_df[face_df['prim_char_flag'] == 1].index.tolist()
len(prim_char_indices)

3163

In [10]:
prim_large_indices = []

for index in prim_char_indices:
    if face_df.loc[index].face_sizes[0] > 4:
        prim_large_indices.append(index)

In [11]:
len(prim_large_indices)

1747

In [12]:
prim_large_indices[0:3]

[81, 82, 91]

We now have a list of frames where there's a primary character, and that character's face is above a certain size. Next, we'll want to use this list to copy these frames into a new folder. We'll put them in `/filtered_character/primary_character/`.

In this directory, we'll have two folders: `positive` and `negative`, for images that fit our criteria, and images that don't.

In [13]:
%ls ../filtered_data/

primary_character/


In [14]:
%ls ../filtered_data/primary_character/

negative/  positive/
